Scarlett Abalco

# JurisRAG-EC Hybrid Legal Intelligence Architecture (HRAG-LegalEC)

Arquitectura híbrida de inteligencia jurídica basada en generación aumentada por recuperación (RAG) — combinando un índice BM25 + E5/FAISS, re-ranker CrossEncoder y modelo generativo T5 con apoyos por regex y NER para extracción estructurada.

# BLOQUE 0 — Setup robusto + utilitarios (dirs, logging, reanudación)

Se preparó el entorno de ejecución instalando dependencias (fitz/pymupdf, rank-bm25, sentence-transformers, faiss-cpu, transformers, accelerate, peft, bitsandbytes) y se definió la estructura de carpetas y artefactos: /content/outputs/, el corpus jurisrag_corpus.parquet, y el directorio /content/outputs/embeddings/ con rutas para passage_emb.npy, faiss.index y meta.parquet. Además, se declararon utilitarios de registro y reanudación (log, save_jsonl_line, already_done_jsonl) para que todo el pipeline sea idempotente y tolerante a cortes, dejando el ambiente consistente antes de parsear o indexar.

In [1]:
# Las librerias
!pip -q install "pymupdf<1.24" unidecode pandas rank-bm25 faiss-cpu sentence-transformers \
                transformers accelerate peft bitsandbytes

import os, json, re, unicodedata, time, random, math, gc, sys
from pathlib import Path
from datetime import datetime
import pandas as pd
from tqdm.auto import tqdm

# Carpetas y helpers
ROOT = Path("/content")
OUT  = ROOT / "outputs"; OUT.mkdir(exist_ok=True)
CORPUS_PARQUET = OUT / "jurisrag_corpus.parquet"
EMB_DIR        = OUT / "embeddings"; EMB_DIR.mkdir(exist_ok=True)
INDEX_FAISS    = EMB_DIR / "faiss.index"
EMB_NPY        = EMB_DIR / "passage_emb.npy"
META_PARQUET   = EMB_DIR / "meta.parquet"
PRED_JSONL     = OUT / "predicciones.jsonl"
PRED_V2_JSONL  = OUT / "predicciones_v2.jsonl"
EVAL_CSV       = OUT / "errores_detalle.csv"
REPORT_SUMMARY = OUT / "resumen_metricas.csv"

def log(msg):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def save_jsonl_line(path: Path, obj: dict):
    with open(path, "a", encoding="utf-8") as w:
        w.write(json.dumps(obj, ensure_ascii=False) + "\n")

def already_done_jsonl(path: Path, key="expediente_id"):
    done=set()
    if path.exists():
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    done.add(json.loads(line).get(key,""))
                except: pass
    return done


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.2 MB/s eta 0:00:00


# BLOQUE 1 — Parsing jurídico + ventanas con solape (reanudable)

Se ingestaron los 30 expedientes y la normativa (Constitución, COIP, COGEP), se segmentaron por secciones (“RAZÓN”, “SENTENCIA”, “AUTO”, “PROVIDENCIA”, etc.) y por ventanas de texto. El pipeline normalizó encabezados y guardó el corpus estructurado en Parquet en /content/outputs/jurisrag_corpus.parquet con ~20 k chunks, además de un resumen de frecuencias por título de sección. El objetivo de este bloque fue dejar una base tabular consistente y repetible para indexación y evaluación posteriores.

In [2]:
import fitz

# Detección de PDFs
PDFS = sorted(ROOT.glob("Expediente ejemplo *.pdf"))
NORM = [ROOT/"Constitucion.pdf", ROOT/"COIP.pdf", ROOT/"COGEP.pdf"]
log(f"Expedientes detectados: {len(PDFS)}")
log(f"Normativa detectada: {[p.name for p in NORM if p.exists()]}")

# Parámetros de chunking
CHUNK_SIZE, CHUNK_OVERLAP, MIN_SECTION_LEN = 1200, 200, 200
SECTION_HEADERS = [
    r"\bSENTENCIA\b", r"\bPROVIDENCIA\b", r"\bRESUELVE?\b", r"\bAUTO\b", r"\bAUTO DE [A-ZÁÉÍÓÚÑ ]+\b",
    r"\bRESOLUCI[ÓO]N\b", r"\bRAZ[ÓO]N\b", r"\bVISTOS\b", r"\bANTECEDENTES\b", r"\bCONSIDERANDO\b"
]

def clean_text(s:str)->str:
    s = s.replace("\x00"," ")
    s = re.sub(r"[ \t]+"," ", s)
    s = re.sub(r"\n{3,}","\n\n", s)
    return unicodedata.normalize("NFKC", s).strip()

def split_by_sections(text:str):
    pat = "(" + "|".join(SECTION_HEADERS) + ")"
    parts = re.split(pat, text, flags=re.IGNORECASE)
    if len(parts) <= 1:
        return [{"title":"DOCUMENTO","body":text}]
    secs, title, buf = [], None, []
    for piece in parts:
        if re.fullmatch(pat, piece, flags=re.IGNORECASE):
            if title or buf:
                body = clean_text("".join(buf))
                if len(body) >= MIN_SECTION_LEN:
                    secs.append({"title":(title or "DOCUMENTO").upper(), "body":body})
            title, buf = piece.strip().upper(), []
        else:
            buf.append(piece)
    if title or buf:
        body = clean_text("".join(buf))
        if len(body) >= MIN_SECTION_LEN:
            secs.append({"title":(title or "DOCUMENTO").upper(), "body":body})
    return secs

def chunk_with_overlap(s:str, size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    s = s.strip()
    if len(s) <= size: return [s]
    out, i = [], 0
    while i < len(s):
        j = min(len(s), i+size)
        out.append(s[i:j])
        if j == len(s): break
        i = max(0, j-overlap)
    return out

def read_pdf(path:Path):
    doc = fitz.open(str(path))
    pages = [clean_text(p.get_text("text") or "") for p in doc]
    doc.close()
    return [t for t in pages if t.strip()]

if CORPUS_PARQUET.exists():
    log(f"Saltando parsing: ya existe {CORPUS_PARQUET}")
    df = pd.read_parquet(CORPUS_PARQUET)
else:
    records, rid = [], 0
    for pdf in tqdm(PDFS, desc="Parseando expedientes"):
        pages = read_pdf(pdf)
        if not pages:
            log(f"[WARN] sin texto: {pdf.name}")
            continue
        full_text = "\n".join([f"[PAG {i+1}]\n{t}" for i,t in enumerate(pages)])
        sections = split_by_sections(full_text)
        for si, sec in enumerate(sections):
            for ci, ch in enumerate(chunk_with_overlap(sec["body"])):
                records.append({
                    "rid": rid,
                    "file": pdf.name,
                    "section_title": sec["title"],
                    "sec_idx": si,
                    "chunk_idx": ci,
                    "chunk_text": ch
                }); rid += 1
    df = pd.DataFrame(records)
    df.to_parquet(CORPUS_PARQUET, index=False)
    log(f"Guardado corpus en {CORPUS_PARQUET} | chunks: {len(df)}")

# Verificación rápida
display(df.groupby("section_title")["rid"].count().sort_values(ascending=False).head(10))


[06:17:58] Expedientes detectados: 30
[06:17:58] Normativa detectada: ['Constitucion.pdf', 'COIP.pdf', 'COGEP.pdf']


Parseando expedientes:   0%|          | 0/30 [00:00<?, ?it/s]

[06:18:35] Guardado corpus en /content/outputs/jurisrag_corpus.parquet | chunks: 20637


,rid
section_title,
RAZÓN,4619
RAZON,4398
SENTENCIA,4099
AUTO,2928
ANTECEDENTES,1384
RESOLUCIÓN,1216
PROVIDENCIA,993
VISTOS,462
CONSIDERANDO,321


# BLOQUE 2 — Índice híbrido (BM25 + E5/FAISS) con checkpointing.

Se construyó el índice híbrido de recuperación: BM25 en memoria con un tokenizador regex para todos los chunk_text, y embeddings densos con intfloat/multilingual-e5-base, normalizados y chequeados por lotes con guardados intermedios a passage_emb.npy. Con esos vectores se creó un índice FAISS (dot-product) y se persistió faiss.index junto a meta.parquet; luego se implementaron los buscadores bm25_search, dense_search y la fusión por Reciprocal Rank Fusion, además de hybrid_search_filtered por expediente. El bloque cerró con un sanity-check que imprime hits y confirmó en logs “BM25 listo” y “Embeddings/FAISS guardados”, validando que la recuperación está operativa para los siguientes bloques.

In [3]:
# ===========================
# BLOQUE 2 — ÍNDICE HÍBRIDO
# BM25 (sparse) + E5 (dense) + FAISS
# Checkpointing: guarda embeddings (.npy) e índice (.index)
# ===========================
import re, numpy as np, faiss, os
from pathlib import Path
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from tqdm.auto import tqdm

# Rutas de salida coherentes con bloque 0/1
from datetime import datetime
def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

OUT = Path("/content/outputs"); OUT.mkdir(exist_ok=True)
EMB_DIR       = OUT / "embeddings"; EMB_DIR.mkdir(exist_ok=True)
EMB_NPY       = EMB_DIR / "passage_emb.npy"
INDEX_FAISS   = EMB_DIR / "faiss.index"
META_PARQUET  = EMB_DIR / "meta.parquet"

# 2.1 BM25 (rápido en RAM)
def tok(s: str):
    return re.findall(r"[a-záéíóúñü0-9]+", s.lower(), flags=re.IGNORECASE)

bm25 = BM25Okapi([tok(t) for t in df["chunk_text"].tolist()])
log("BM25 listo.")

# 2.2 Embeddings densos (E5 multilingüe) con reanudación
EMB_MODEL = "intfloat/multilingual-e5-base"  # robusto multilingüe
emb = SentenceTransformer(EMB_MODEL)

if EMB_NPY.exists() and INDEX_FAISS.exists() and META_PARQUET.exists():
    log("Saltando codificación: ya existen NPY + FAISS.")
    P = np.load(EMB_NPY)
    index = faiss.read_index(str(INDEX_FAISS))
else:
    texts = [f"passage: {t}" for t in df["chunk_text"].tolist()]
    bs = 128                                       # ajusta si tu GPU es pequeña
    chunks = []
    for i in tqdm(range(0, len(texts), bs), desc="Embeddings E5"):
        batch = texts[i:i+bs]
        v = emb.encode(batch, normalize_embeddings=True).astype("float32")
        chunks.append(v)
        # checkpoint cada ~50 batches
        if (i//bs) % 50 == 0 and i>0:
            tmp = np.vstack(chunks)
            np.save(EMB_NPY, tmp)
            log(f"checkpoint guardado: {tmp.shape}")
    P = np.vstack(chunks)
    np.save(EMB_NPY, P)
    index = faiss.IndexFlatIP(P.shape[1]); index.add(P)
    faiss.write_index(index, str(INDEX_FAISS))
    df[["rid","file","section_title","chunk_idx"]].to_parquet(META_PARQUET, index=False)
    log(f"Embeddings/FAISS guardados en {EMB_DIR}")

# 2.3 Buscadores
def bm25_search(query, topk=50):
    scores = bm25.get_scores(tok(query))
    idx = np.argsort(-scores)[:topk]
    return [(int(i), float(scores[i])) for i in idx]

def dense_search(query, topk=50):
    qv = emb.encode([f"query: {query}"], normalize_embeddings=True).astype("float32")
    D, I = index.search(qv, topk)
    return [(int(i), float(d)) for i,d in zip(I[0], D[0])]

def rrf(bm_hits, de_hits, k=60, c=60):
    ranks = defaultdict(float)
    for r,(i,_) in enumerate(bm_hits[:k], 1): ranks[i] += 1/(c+r)
    for r,(i,_) in enumerate(de_hits[:k], 1): ranks[i] += 1/(c+r)
    return sorted(ranks.items(), key=lambda x: x[1], reverse=True)

def hybrid_search(query, topk=10):
    boost = bool(re.search(r"sanci[oó]n|pena|multa", query, re.IGNORECASE))
    fused=[]
    for idx,score in rrf(bm25_search(query,80), dense_search(query,80), k=80):
        row = df.iloc[idx]
        if boost and row["section_title"] in ("PROVIDENCIA","SENTENCIA","RESUELVE"): score += 0.15
        rec = row.to_dict(); rec.update({"idx": idx, "rrf_score": score})
        fused.append(rec)
        if len(fused) >= 4*topk:  # corta temprano
            break
    return sorted(fused, key=lambda x: x["rrf_score"], reverse=True)[:topk]

# Búsqueda filtrada por expediente (útil para extracción por archivo)
def hybrid_search_filtered(query, filter_file, topk=12):
    fused = rrf(bm25_search(query,80), dense_search(query,80), k=80)
    out=[]
    for idx,score in fused:
        row = df.iloc[idx]
        if row["file"] != filter_file: continue
        if row["section_title"] in ("PROVIDENCIA","SENTENCIA","RESUELVE"): score += 0.10
        rec = row.to_dict(); rec.update({"idx": idx, "rrf_score": score})
        out.append(rec)
        if len(out) >= topk: break
    return out

# 2.4 Sanity check de recuperación
for q in ["sanción o multa", "juez que firma", "artículos aplicados"]:
    hits = hybrid_search(q, topk=5)
    print("\n>>>", q)
    for h in hits:
        print(f"[{h['file']} | {h['section_title']} | chunk {h['chunk_idx']}] rrf={h['rrf_score']:.3f}")
        print(h["chunk_text"][:160].replace("\n"," ") + "…")


[06:19:13] BM25 listo.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Embeddings E5:   0%|          | 0/162 [00:00<?, ?it/s]

[06:21:55] checkpoint guardado: (6528, 768)
[06:24:12] checkpoint guardado: (12928, 768)
[06:26:23] checkpoint guardado: (19328, 768)
[06:26:49] Embeddings/FAISS guardados en /content/outputs/embeddings

>>> sanción o multa
[Expediente ejemplo 12.pdf | SENTENCIA | chunk 0] rrf=0.175
escrita, deberá contener: No. 10.- La suspensión condicional de la pena y señalamiento del plazo dentro  del cual se pagará la multa, cuando corresponda.” Dejan…
[Expediente ejemplo 13.pdf | SENTENCIA | chunk 0] rrf=0.174
que se encuentra debidamente ejecutoriada por el Ministerio de la  Ley, informo que de la revisión del proceso y del Sistema Satje NO consta que la sentenciada …
[Expediente ejemplo 12.pdf | SENTENCIA | chunk 0] rrf=0.172
ya que la suspensión condicional de la pena corresponde  única y exclusivamente a la pena privativa de libertad; conforme el mismo Art. 70 del Código Orgánico I…
[Expediente ejemplo 1.pdf | SENTENCIA | chunk 0] rrf=0.171
. 14.1.3.- MULTA DE CINCO SALARIOS BÁSICOS UNIFICAD

# BLOQUE 3 — Generador T5 + QA con citas (smoke test)

Se construyó la capa de recuperación híbrida: un BM25 local y un índice de embeddings E5 en FAISS, con fusión por Reciprocal Rank Fusion. El cuaderno mostró las barras de descarga de E5 y la escritura de artefactos en /content/outputs/embeddings, y luego ejemplos de “hits” con rrf_score para preguntas de control (“sanción/multa impuesta”, “juez que firma”). El bloque confirma que el retriever devuelve pasajes de “SENTENCIA”/“PROVIDENCIA” coherentes y que la función rag_qa puede generar una respuesta mostrando las citas de contexto.

In [4]:
# === BLOQUE 3 === Generador T5 + QA con citas
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch, re

GEN = "google/flan-t5-base"
tok_g = AutoTokenizer.from_pretrained(GEN)
gen   = AutoModelForSeq2SeqLM.from_pretrained(GEN).to("cuda" if torch.cuda.is_available() else "cpu")

QA_SYSTEM = ("Eres un asistente jurídico ecuatoriano. Responde SOLO con base en el CONTEXTO. "
             "Si no está en el contexto, responde: 'No se encuentra en los documentos proporcionados'. "
             "Incluye SIEMPRE citas: [archivo | ACTO | chunk].")

def build_context(passages):
    return "\n\n".join([f"[{p['file']} | {p['section_title']} | chunk {p['chunk_idx']}]\n{p['chunk_text']}" for p in passages])

def generate_t5(prompt, max_new_tokens=256, greedy=True):
    dev = "cuda" if torch.cuda.is_available() else "cpu"
    with torch.inference_mode():
        x = tok_g(prompt, return_tensors="pt", truncation=True, max_length=2048).to(dev)
        y = gen.generate(**x, max_new_tokens=max_new_tokens, do_sample=not greedy,
                         temperature=0.3, top_p=0.9)
        return tok_g.decode(y[0], skip_special_tokens=True).strip()

def rag_qa(question, topk_ret=12, topk_ctx=4):
    ctx = hybrid_search(question, topk=topk_ret)[:topk_ctx]
    prompt = f"{QA_SYSTEM}\n\nPregunta: {question}\n\nCONTEXTO:\n{build_context(ctx)}\n\nRespuesta:"
    ans = generate_t5(prompt, max_new_tokens=220, greedy=True)
    return {"answer": ans, "citations": [(c['file'], c['section_title'], c['chunk_idx']) for c in ctx]}

# Smoke test rápido
for q in ["¿Cuál es la sanción o multa impuesta?", "¿Quién es el juez o jueza que firma o resuelve?"]:
    r = rag_qa(q)
    print("\n>>>", q)
    print(r["answer"])
    print("Citas:", r["citations"])


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



>>> ¿Cuál es la sanción o multa impuesta?
[EXPENDIENTE ejemplo 12.pdf | SENTENCIA | chunk 0] escrita, deberá contener: No. 10.- La suspensión condicional de la pena y sealamiento del plazo dentro del cual se pagará la multa, cuando corresponda.” Dejando aclarado el hecho de haber dispuesto se justifica el pago de la multa impuesta como parte de la ejecución de la [Expediente ejemplo 23.pdf | SENTENCIA | chunk 0] . 4. Por ser un crime de inviolabilidad a la vida, dentro la Reparación integral a la vctima, se ordena a la sentenciado pague como indemnización le
Citas: [('Expediente ejemplo 12.pdf', 'SENTENCIA', 0), ('Expediente ejemplo 23.pdf', 'SENTENCIA', 0), ('Expediente ejemplo 22.pdf', 'SENTENCIA', 0), ('Expediente ejemplo 12.pdf', 'SENTENCIA', 5)]

>>> ¿Quién es el juez o jueza que firma o resuelve?
MONTENEGRO SUAREZ GUILLERMO ROBERTO COMPARECIENTE MAROTO SANCHEZ MILTON IVAN JUEZ PONENTE 18/02/2025 16:17 Responde SOLO 'No se encuentra en los documentos proporcionados'.
Citas: [('Ex

# Bloque 4 – Extracción por campo (v2)

Se ejecutó la extracción campo-a-campo con un T5 generativo en modo estricto, apoyado por un fallback regex y contexto focalizado por el retriever. Los campos extraídos fueron numero_proceso, juez, fiscal, tipo_acto, fecha_acto, decision y articulos_citados; se reanudó de forma segura y se guardó cada expediente en /content/outputs/predicciones_v3.jsonl. La corrida quedó registrada con “Reanudar v3: ya procesados 0 de 30” y cerró con “Predicciones v3 guardadas…”, tardando ~5 h 39 min en tu sesión.

In [9]:
# === BLOQUE 4 — Extracción por campo v3 (T5 + regex, reanudable) ===
import json, re
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

# Guardamoss en predicciones_v3.jsonl y el dict se llamará "ie_fields_v3"
PRED_V3_JSONL = Path("/content/outputs/predicciones_v3.jsonl")

# --- Regex de respaldo coherentes con nuestros campos ---
re_proceso = re.compile(r"(?:N[ºo]\s*|No\.?\s*|N°\s*|Nro\.?\s*|Número\s*de\s*proceso\s*[:\-]?\s*)(\d{2,6}[-–]\d{4}(?:[-–]\d{2,6})?)")
re_juez    = re.compile(r"\b(JUEZ[A]?(?:\s+PONENTE)?|JUEZA)\b[:\-]?\s*([A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑÜü\s\.\-']{3,})")
re_fiscal  = re.compile(r"\b(AGENTE\s+FISCAL|FISCAL[A]?)\b[:\-]?\s*([A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑÜü\s\.\-']{3,})")
re_fecha   = re.compile(r"\b(\d{4}-\d{2}-\d{2}|\d{1,2}/\d{1,2}/\d{4})\b")
re_art     = re.compile(r"\bArt(?:í|i)?\.?\s*(\d{1,3})\s*(?:del|de la)?\s*(COIP|Constituci[oó]n|Constitución|COGEP)?", re.IGNORECASE)

# --- Consultas de recuperación por campo (para sesgar a pasajes relevantes de ese archivo) ---
def field_queries(campo):
    return {
        "numero_proceso":  ["número de proceso", "N° de proceso", "Nro. proceso", "No.", "Número de proceso"],
        "juez":            ["juez ponente", "jueza", "ADMINISTRANDO JUSTICIA", "quien firma", "firma"],
        "fiscal":          ["agente fiscal", "fiscal del caso", "FISCALÍA"],
        "tipo_acto":       ["SENTENCIA", "PROVIDENCIA", "RESUELVE", "AUTO"],
        "fecha_acto":      ["fecha", "en la ciudad de", "a los"],
        "decision":        ["RESUELVE", "SENTENCIA", "FALLA", "DECIDE"],
        "articulos_citados":["Art.", "artículos", "COIP", "Constitución", "COGEP"]
    }.get(campo, [campo])

# --- Recuperación limitada al mismo archivo + deduplicación + priorización por secciones clave ---
def field_context(file_name, campo, topk=16):
    hits=[]
    for q in field_queries(campo):
        hits.extend(hybrid_search_filtered(q, filter_file=file_name, topk=topk))
    seen=set(); out=[]
    # priorizamos SENTENCIA/PROVIDENCIA/RESUELVE y score RRF
    for h in sorted(hits, key=lambda x: (x["section_title"] in ("SENTENCIA","PROVIDENCIA","RESUELVE"), x["rrf_score"]), reverse=True):
        if h["idx"] in seen:
            continue
        seen.add(h["idx"]); out.append(h)
    return out[:min(6, len(out))]

def concat_ctx(passages):
    return "\n\n".join([f"[{p['file']} | {p['section_title']} | chunk {p['chunk_idx']}]\n{p['chunk_text']}" for p in passages]) if passages else ""

# --- Regex fallback (cuando T5 no devuelve un valor usable) ---
def regex_fallback(campo, text):
    if not text:
        return None
    if campo=="numero_proceso":
        m=re_proceso.search(text);
        return m.group(1) if m else None
    if campo=="juez":
        m=re_juez.search(text);
        return m.group(2).strip() if m else None
    if campo=="fiscal":
        m=re_fiscal.search(text);
        return m.group(2).strip() if m else None
    if campo=="fecha_acto":
        m=re_fecha.search(text);
        return m.group(1).replace("/","-") if m else None
    if campo=="articulos_citados":
        arts=[]
        for m in re_art.finditer(text):
            num=m.group(1); code=m.group(2) or ""
            arts.append(f"Art. {num} {code}".strip())
        if arts:
            seen=set(); uniq=[]
            for a in arts:
                if a not in seen:
                    uniq.append(a); seen.add(a)
            return uniq[:15]
    return None

# --- Pregunta a T5 por campo, con plantillas estrictas (y greedy=True para determinismo) ---
def ask_field(file_name, campo):
    ctx = field_context(file_name, campo, topk=16)
    ctx_text = concat_ctx(ctx)
    if campo=="articulos_citados":
        prompt = (
            "Devuelve SOLO una lista JSON de artículos citados (p.ej. \"Art. 70 COIP\"). "
            "Si no hay, devuelve [].\n\nCONTEXTO:\n"
            f"{ctx_text}\n\nJSON:"
        )
        ans = generate_t5(prompt, max_new_tokens=180, greedy=True)
        try:
            js = json.loads(ans)
            if isinstance(js, list):
                return js
        except:
            pass
        return None
    else:
        prompt = (
            f"Devuelve SOLO el valor del campo '{campo}' (una línea). "
            "Si no está, devuelve null.\n\nCONTEXTO:\n"
            f"{ctx_text}\n\nValor:"
        )
        ans = generate_t5(prompt, max_new_tokens=100, greedy=True).strip()
        # Normalizamos valores vacíos típicos
        if ans.lower() in ["null","nulo","none","no se encuentra","no se encuentra en los documentos proporcionados"]:
            ans=None
        return ans

# --- Helpers de guardado / reanudación ---
def save_jsonl_line(path: Path, obj: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "a", encoding="utf-8") as w:
        w.write(json.dumps(obj, ensure_ascii=False) + "\n")

def already_done_jsonl(path: Path, key="expediente_id"):
    done=set()
    if path.exists():
        for line in open(path,"r",encoding="utf-8"):
            try: done.add(json.loads(line).get(key,""))
            except: pass
    return done

# === Extracción reanudable ===
files = df["file"].unique().tolist()
hechos = already_done_jsonl(PRED_V3_JSONL)
log(f"Reanudar v3: ya procesados {len(hechos)} de {len(files)}")

for f in tqdm(files, desc="Extracción v3 por expediente (T5 + regex)"):
    eid = re.sub(r"\.pdf$","", f, flags=re.IGNORECASE).replace(" ","_")
    if eid in hechos:
        continue
    campos = ["numero_proceso","juez","fiscal","tipo_acto","fecha_acto","decision","articulos_citados"]
    res={}
    for c in campos:
        v = ask_field(f, c)
        if v in (None, [], ""):
            # fallback por regex con más contexto
            text = concat_ctx(field_context(f, c, topk=24))
            v = regex_fallback(c, text) or v
        res[c]=v
    save_jsonl_line(PRED_V3_JSONL, {"expediente_id": eid, "file": f, "ie_fields_v3": res})

log(f"✅ Predicciones v3 guardadas en: {PRED_V3_JSONL}")


[07:55:06] Reanudar v3: ya procesados 0 de 30


Extracción v3 por expediente (T5 + regex):   0%|          | 0/30 [00:00<?, ?it/s]

[08:00:46] ✅ Predicciones v3 guardadas en: /content/outputs/predicciones_v3.jsonl


# BLOQUE 5 — Evaluación v3 (EM/F1 + CSVs)

Se cargaron las respuestas oficiales y se midieron EM y F1 por campo, además de guardar los errores con sus “pred” y “ref”. El cuaderno generó /content/outputs/resumen_metricas_v3.csv y /content/outputs/errores_detalle_v3.csv, y listó muestras de discrepancias por campo; en tus capturas, el EM/F1 global de v3 quedó muy bajo (en un corte fue ~0.014 y en el corte “oficial” posterior ~0.005). El paneo de errores muestra que número_proceso, juez, fiscal y fecha_acto aparecen vacíos o con ruido, y que articulos_citados tiene algunas coincidencias parciales.

In [10]:
# === BLOQUE 5 — Evaluación v3 (EM/F1 + CSVs) ===
import json, re, pandas as pd
from pathlib import Path
from collections import Counter

PRED_FILE = Path("/content/outputs/predicciones_v3.jsonl")   # generado en el Bloque 4 (v3)
REF_FILE  = Path("/content/respuestas_oficiales.jsonl")      # ground-truth (abogada, que extrajo las respuestas correctas)
OUT_SUM   = Path("/content/outputs/resumen_metricas_v3.csv")
OUT_ERR   = Path("/content/outputs/errores_detalle_v3.csv")

assert PRED_FILE.exists(), f"No existe {PRED_FILE}. Ejecuta antes el Bloque 4."


if not REF_FILE.exists():
    template = []
    for line in open(PRED_FILE, "r", encoding="utf-8"):
        obj = json.loads(line)
        template.append({
            "expediente_id": obj["expediente_id"],
            "ie_fields": {
                "numero_proceso": "",
                "juez": "",
                "fiscal": "",
                "tipo_acto": "",
                "fecha_acto": "",
                "decision": "",
                "articulos_citados": []
            }
        })
    tmp_path = Path("/content/respuestas_oficiales_template.jsonl")
    with open(tmp_path, "w", encoding="utf-8") as w:
        for row in template:
            w.write(json.dumps(row, ensure_ascii=False) + "\n")
    print(f"⚠️ No se encontró {REF_FILE}. Se generó plantilla en: {tmp_path}\n"
          "Rellénala y guarda como /content/respuestas_oficiales.jsonl. "
          "Luego vuelve a ejecutar este bloque.")
else:
    # ---------- utilidades ----------
    def normalize(s):
        if s is None: return ""
        if isinstance(s, list):
            s = ", ".join([str(x) for x in s])
        s = str(s).strip()
        s = re.sub(r"\s+"," ", s)
        return s

    def tokens(s):
        s = normalize(s).lower()
        s = re.sub(r"[^0-9a-záéíóúñü]+", " ", s)
        return s.split()

    def f1_score(pred, ref):
        # soporta string o lista
        if isinstance(pred, list): pred = "; ".join([normalize(x) for x in pred])
        if isinstance(ref,  list): ref  = "; ".join([normalize(x) for x in ref])
        pt, rt = tokens(pred), tokens(ref)
        if len(pt)==0 and len(rt)==0: return 1.0
        if len(pt)==0 or len(rt)==0:  return 0.0
        inter = Counter(pt) & Counter(rt)
        num = sum(inter.values())
        if num==0: return 0.0
        prec = num / len(pt)
        rec  = num / len(rt)
        return (2*prec*rec)/(prec+rec) if (prec+rec)>0 else 0.0

    def exact_match(pred, ref):
        if isinstance(pred, list): pred = "; ".join([normalize(x) for x in pred])
        if isinstance(ref,  list): ref  = "; ".join([normalize(x) for x in ref])
        return int(normalize(pred) == normalize(ref))

    # ---------- carga ----------
    preds = [json.loads(l) for l in open(PRED_FILE, "r", encoding="utf-8")]
    refs  = { x["expediente_id"]: x for x in [json.loads(l) for l in open(REF_FILE, "r", encoding="utf-8")] }

    CAMPOS = ["numero_proceso","juez","fiscal","tipo_acto","fecha_acto","decision","articulos_citados"]

    rows=[]
    for p in preds:
        eid = p["expediente_id"]
        pv  = p.get("ie_fields_v3", {})                 # <- diccionario v3
        rv  = refs.get(eid, {}).get("ie_fields", {})    # <- referencia de la abogada
        for c in CAMPOS:
            pred = pv.get(c, None)
            ref  = rv.get(c, None)
            f1   = f1_score("" if pred is None else pred, "" if ref is None else ref)
            em   = exact_match("" if pred is None else pred, "" if ref is None else ref)
            rows.append({"expediente_id":eid, "campo":c, "pred":pred, "ref":ref, "EM":em, "F1":f1})

    dfm = pd.DataFrame(rows)

    # ---------- métricas ----------
    agg = dfm.groupby("campo")[["EM","F1"]].mean().sort_values("F1", ascending=False)
    print("=== Métricas por campo (v3) ===")
    display(agg)

    print("\n=== Resumen general (v3) ===")
    print(f"EM global: {agg['EM'].mean():.3f} | F1 global: {agg['F1'].mean():.3f}")

    # ---------- guardado ----------
    OUT_SUM.parent.mkdir(parents=True, exist_ok=True)
    agg.to_csv(OUT_SUM)
    dfm[(dfm["EM"]==0)][["expediente_id","campo","pred","ref","F1"]]\
       .sort_values(["campo","F1"]).to_csv(OUT_ERR, index=False)

    print(f"[OK] CSVs guardados:\n - {OUT_SUM}\n - {OUT_ERR}")

    # ---------- muestras de error (para iterar prompts/regex) ----------
    for k in CAMPOS:
        muestra = dfm[(dfm["campo"]==k) & (dfm["EM"]==0)][["expediente_id","campo","pred","ref","F1"]].head(5)
        if len(muestra):
            print(f"\n=== Muestras de error: {k} ===")
            display(muestra)


=== Métricas por campo (v3) ===


,EM,F1
campo,,
articulos_citados,0.066667,0.066667
fecha_acto,0.033333,0.033333
decision,0.000000,0.000000
fiscal,0.000000,0.000000
juez,0.000000,0.000000
numero_proceso,0.000000,0.000000
tipo_acto,0.000000,0.000000



=== Resumen general (v3) ===
EM global: 0.014 | F1 global: 0.014
[OK] CSVs guardados:
 - /content/outputs/resumen_metricas_v3.csv
 - /content/outputs/errores_detalle_v3.csv

=== Muestras de error: numero_proceso ===


,expediente_id,campo,pred,ref,F1
0,Expediente_ejemplo_1,numero_proceso,1. nte a la causa en Memorando N° DP23-SMCPJSD...,None,0.0
7,Expediente_ejemplo_10,numero_proceso,a.-) Incorporórese el proceso anexos y escrito...,None,0.0
14,Expediente_ejemplo_11,numero_proceso,En mi calidad de juez integrante del Tribunal ...,None,0.0
21,Expediente_ejemplo_12,numero_proceso,[Expediente ejemplo 12.pdf | SENTENCIA | chunk...,None,0.0
28,Expediente_ejemplo_13,numero_proceso,Devuelve SOLO el valor del campo 'numero_proce...,None,0.0



=== Muestras de error: juez ===


...

,expediente_id,campo,pred,ref,F1
4,Expediente_ejemplo_1,fecha_acto,Devuelve SOLO el valor del campo 'fecha_acto' ...,None,0.0
11,Expediente_ejemplo_10,fecha_acto,Devuelve SOLO el valor del campo 'fecha_acto' ...,None,0.0
18,Expediente_ejemplo_11,fecha_acto,a aproximación del acero a a a a a a a a a a a...,None,0.0
25,Expediente_ejemplo_12,fecha_acto,el acuerdo del acuerdo del acuerdo del acuerdo...,None,0.0
32,Expediente_ejemplo_13,fecha_acto,"lava Chávez Rosa Elena, a testimonio de compra...",None,0.0



=== Muestras de error: decision ===


,expediente_id,campo,pred,ref,F1
5,Expediente_ejemplo_1,decision,ltimo el valor del campo 'decision' (una lnea)...,None,0.0
12,Expediente_ejemplo_10,decision,el valor del campo 'decision' (una lnea). Si n...,None,0.0
19,Expediente_ejemplo_11,decision,Devuelve SOLO el valor del campo 'decision' (u...,None,0.0
26,Expediente_ejemplo_12,decision,Devuelve SOLO el valor del campo 'decision' (u...,None,0.0
33,Expediente_ejemplo_13,decision,Devuelve SOLO el valor del campo 'decision' (u...,None,0.0



=== Muestras de error: articulos_citados ===


,expediente_id,campo,pred,ref,F1
6,Expediente_ejemplo_1,articulos_citados,"[Art. 76, Art. 369, Art. 282, Art. 195 Constit...",None,0.0
13,Expediente_ejemplo_10,articulos_citados,"[Art. 86 Constitución, Art. 25, Art. 400, Art....",None,0.0
20,Expediente_ejemplo_11,articulos_citados,"[Art. 220, Art. 42, Art. 1, Art. 69, Art. 60, ...",None,0.0
27,Expediente_ejemplo_12,articulos_citados,"[Art. 622, Art. 56, Art. 64, Art. 12, Art. 81]",None,0.0
34,Expediente_ejemplo_13,articulos_citados,"[Art. 454 COIP, Art. 454, Art. 70, Art. 297 co...",None,0.0


# BLOQUE 6 — Re-ranker + NER

Se añadió re-ranking con CrossEncoder y un NER en español para reforzar spans de nombres, procesos y artículos, manteniendo el mismo esquema de persistencia y reanudación. El notebook cargó correctamente mrm8488/bert-spanish-cased-finetuned-ner en GPU (“Device set to use cuda:0”), re-rankeó y guardó /content/outputs/predicciones_v4.jsonl con “30/30 [07:49<…]”, sin tocar la lógica de evaluación. La intención de este bloque fue aumentar la precisión de spans antes de la decisión del generador, sin cambiar el contrato de salida.

In [13]:
# === BLOQUE 6 — Re-ranker + extractor NER (v4, reanudable; sin tokens HF) ===
# Mejora sobre v3: ordena pasajes con CrossEncoder y usa NER HF español/multilingüe cuando esté disponible.

!pip -q install sentence-transformers "transformers>=4.41.0"

from sentence_transformers import CrossEncoder
from transformers import pipeline
import torch, re, json
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm

def log(msg):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Re-ranker público (no requiere token)
log("Cargando re-ranker CrossEncoder…")
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", device=DEVICE)

# --- Cargador NER con fallbacks a modelos PÚBLICOS (sin token).
# Intenta en orden y si ninguno se puede bajar, desactiva NER.
def load_ner_pipeline():
    candidates = [
        # Español (público)
        "mrm8488/bert-spanish-cased-finetuned-ner",
        # Multilingüe (público)
        "Babelscape/wikineural-multilingual-ner",
        # Otro multilingüe (público). Si falla alguno, probamos el siguiente.
        "Davlan/xlm-roberta-base-ner-hrl",
    ]
    for model_id in candidates:
        try:
            log(f"Intentando cargar NER: {model_id} …")
            ner = pipeline(
                task="token-classification",
                model=model_id,
                aggregation_strategy="simple",
                device=0 if torch.cuda.is_available() else -1,
            )
            log(f"NER cargado: {model_id}")
            return ner
        except Exception as e:
            log(f"No se pudo cargar {model_id}. Motivo: {e}")
    log("⚠️ No se pudo cargar ningún modelo NER público. Continuaré SIN NER (sólo regex).")
    return None

ner = load_ner_pipeline()

# --- regex auxiliares
re_proceso = re.compile(r"(?:N[ºo]\s*|No\.?\s*|N°\s*|Nro\.?\s*|Número\s*de\s*proceso\s*[:\-]?\s*)(\d{2,6}[-–]\d{4}(?:[-–]\d{2,6})?)")
re_fecha   = re.compile(r"\b(\d{4}-\d{2}-\d{2}|\d{1,2}/\d{1,2}/\d{4})\b")
re_art     = re.compile(r"\bArt(?:í|i)?\.?\s*(\d{1,3})\s*(?:del|de la)?\s*(COIP|Constituci[oó]n|Constitución|COGEP)?", re.IGNORECASE)

def field_queries(campo):
    return {
        "numero_proceso": ["número de proceso", "N° de proceso", "Nro. proceso", "No."],
        "juez": ["juez ponente", "jueza", "ADMINISTRANDO JUSTICIA", "quien firma"],
        "fiscal": ["agente fiscal", "fiscal del caso", "fiscalía"],
        "tipo_acto": ["SENTENCIA", "PROVIDENCIA", "RESUELVE", "AUTO"],
        "fecha_acto": ["fecha", "en la ciudad de", "a los"],
        "decision": ["RESUELVE", "SENTENCIA", "FALLA", "DECIDE"],
        "articulos_citados": ["Art.", "artículos", "COIP", "Constitución", "COGEP"]
    }.get(campo, [campo])

# --- Reranking y contexto
def rerank_ctx(query, candidates, keep=10):
    if not candidates:
        return []
    pairs = [(query, c["chunk_text"]) for c in candidates]
    scores = reranker.predict(pairs, convert_to_numpy=True)
    for c, s in zip(candidates, scores):
        c["rerank_score"] = float(s)
    return sorted(candidates, key=lambda x: x["rerank_score"], reverse=True)[:keep]

def field_context_rerank(file_name, campo, topk_ret=48, keep=10):
    cands=[]
    for q in field_queries(campo):
        cands.extend(hybrid_search_filtered(q, filter_file=file_name, topk=topk_ret))
    seen, uniq = set(), []
    for h in cands:
        if h["idx"] in seen: continue
        seen.add(h["idx"]); uniq.append(h)
    # reordenamos con el CrossEncoder
    return rerank_ctx(" ".join(field_queries(campo)) + " " + file_name, uniq, keep=keep)

def concat_ctx(passages):
    return "\n\n".join([f"[{p['file']} | {p['section_title']} | chunk {p['chunk_idx']}]\n{p['chunk_text']}"
                        for p in passages])

# --- Regex y NER fallbacks
def regex_fallback(campo, text):
    if campo=="numero_proceso":
        m=re_proceso.search(text); return m.group(1) if m else None
    if campo=="fecha_acto":
        m=re_fecha.search(text); return m.group(1).replace("/","-") if m else None
    if campo=="articulos_citados":
        arts=[]
        for m in re_art.finditer(text):
            num, code = m.group(1), (m.group(2) or "")
            arts.append(f"Art. {num} {code}".strip())
        if arts:
            seen=set(); out=[]
            for a in arts:
                if a not in seen: out.append(a); seen.add(a)
            return out[:20]
    return None

def ner_person_cleanup(text):
    if ner is None:
        return None
    outs = ner(text[:3000])
    cands = [o["word"].strip() for o in outs if o.get("entity_group","") in {"PER","PERSON"}]
    bad = {"resolución","providencia","sentencia","auto","juzgado","tribunal","fiscalía","unidad"}
    cands = [c for c in cands if not any(b in c.lower() for b in bad)]
    return cands[0] if cands else None

# --- Q/A principal (usa generate_t5 ya definido en Bloque 3)
def ask_field_v4(file_name, campo):
    ctx = field_context_rerank(file_name, campo, topk_ret=48, keep=10)
    ctx_text = concat_ctx(ctx)

    if campo=="articulos_citados":
        prompt = (
            "Devuelve SOLO una lista JSON de artículos citados (ej. \"Art. 70 COIP\"). "
            "Si no hay, devuelve []. NO inventes.\n\nCONTEXTO:\n" + ctx_text + "\n\nJSON:"
        )
        ans = generate_t5(prompt, max_new_tokens=220, greedy=True).strip()
        try:
            js = json.loads(ans)
            if isinstance(js, list): return js[:20]
        except:
            pass
        return regex_fallback(campo, ctx_text)

    prompt = f"Devuelve SOLO el valor del campo '{campo}' (una línea). Si no está, devuelve null.\n\nCONTEXTO:\n{ctx_text}\n\nValor:"
    ans = generate_t5(prompt, max_new_tokens=120, greedy=True).strip()
    if ans.lower() in {"null","nulo","none","no se encuentra","no consta"}:
        ans=None
    if not ans:
        v = regex_fallback(campo, ctx_text)
        if v: return v
    if not ans and campo in {"juez","fiscal"}:
        v = ner_person_cleanup(ctx_text)
        if v: return v
    return ans

# --- Ejecución reanudable ---
PRED_V4_JSONL = Path("/content/outputs/predicciones_v4.jsonl")

def already_done_jsonl(path, key="expediente_id"):
    done=set()
    if path.exists():
        for line in open(path,"r",encoding="utf-8"):
            try: done.add(json.loads(line).get(key,""))
            except: pass
    return done

def save_jsonl_line(path, obj):
    with open(path, "a", encoding="utf-8") as w:
        w.write(json.dumps(obj, ensure_ascii=False)+"\n")

files = df["file"].unique().tolist()
hechos = already_done_jsonl(PRED_V4_JSONL)
log(f"Reanudar v4: ya procesados {len(hechos)} de {len(files)}")

for f in tqdm(files, desc="Extracción v4 (rerank + NER)"):
    eid = re.sub(r"\.pdf$","", f, flags=re.IGNORECASE).replace(" ","_")
    if eid in hechos:
        continue
    campos = ["numero_proceso","juez","fiscal","tipo_acto","fecha_acto","decision","articulos_citados"]
    res = {c: ask_field_v4(f, c) for c in campos}
    save_jsonl_line(PRED_V4_JSONL, {"expediente_id": eid, "file": f, "ie_fields_v4": res})

log(f"✅ Predicciones v4 guardadas en {PRED_V4_JSONL}")


[08:23:45] Cargando re-ranker CrossEncoder…
[08:23:46] Intentando cargar NER: mrm8488/bert-spanish-cased-finetuned-ner …


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[08:23:56] NER cargado: mrm8488/bert-spanish-cased-finetuned-ner
[08:23:56] Reanudar v4: ya procesados 0 de 30


Extracción v4 (rerank + NER):   0%|          | 0/30 [00:00<?, ?it/s]

[08:31:46] ✅ Predicciones v4 guardadas en /content/outputs/predicciones_v4.jsonl


# BLOQUE 7 — Evaluación oficial (respuestas reales de la abogada)

Se recalcularon EM y F1 con las respuestas oficiales y se escribió la comparativa en /content/outputs/comparativa_v3_v4.csv. En tus resultados el resumen oficial reporta EM global ≈ 0.005 y F1 global ≈ 0.005, y la tabla “COMPARATIVA” enseña ΔEM_v4-v3 cercano a 0 en casi todos los campos y negativo en artículos_citados (-0.033333 en tu corte), lo que evidencia que, con este conjunto, el re-ranking+NER no desplazó la aguja. Se guardaron también los CSV de detalle por campo para iterar prompts y regex.

In [18]:
# === BLOQUE 7 — Evaluación oficial (solo respuestas reales de la abogada) ===
import json, re, pandas as pd
from pathlib import Path
from datetime import datetime

def log(msg):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

# --- Rutas principales ---
PRED_V4 = Path("/content/outputs/predicciones_v4.jsonl")  # predicciones del modelo
GOLD    = Path("/content/respuestas_oficiales.jsonl")     # respuestas reales de la abogada
PRED_V3 = Path("/content/outputs/predicciones_v3.jsonl")  # para comparar

assert PRED_V4.exists(), "❌ No se encontró /content/outputs/predicciones_v4.jsonl"
assert GOLD.exists(),    "❌ No se encontró /content/respuestas_oficiales.jsonl"

# --- Leer archivos JSONL ---
def read_jsonl(p: Path):
    data=[]
    with open(p, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

pred_v4 = read_jsonl(PRED_V4)
gold    = read_jsonl(GOLD)

# --- Normalización general ---
def norm(x):
    if x is None: return None
    if isinstance(x, list):
        vals = [norm(v) for v in x if v]
        seen=set(); uniq=[]
        for v in vals:
            if v not in seen:
                uniq.append(v); seen.add(v)
        return uniq
    s=str(x).strip().lower()
    s=re.sub(r"\s+"," ",s)
    return s

# --- F1 básico ---
def f1_str(pred, ref):
    if pred is None and ref is None: return 1.0
    if pred is None or ref is None:  return 0.0
    if isinstance(pred, list) or isinstance(ref, list):
        P=set(pred if isinstance(pred, list) else [pred])
        R=set(ref  if isinstance(ref, list)  else [ref])
        tp=len(P & R); fp=len(P-R); fn=len(R-P)
        if tp==0: return 0.0
        prec=tp/(tp+fp); rec=tp/(tp+fn)
        return 2*prec*rec/(prec+rec)
    return float(pred==ref)

# --- Campos evaluados ---
CAMPOS = ["numero_proceso","juez","fiscal","tipo_acto","fecha_acto","decision","articulos_citados"]

# --- Construcción de mapas expediente -> campos ---
def extract_fields(record):
    # Solo se aceptan los campos que la abogada realmente llenó
    for key in ["ie_fields","ie_fields_v2","ie_fields_v3","ie_fields_v4","respuestas","campos"]:
        if key in record and isinstance(record[key], dict):
            return record[key]
    # Si los campos están al nivel raíz (formato manual)
    filtered = {k:v for k,v in record.items() if k in CAMPOS}
    return filtered if filtered else {}

# Respuestas oficiales
gold_map = {}
for g in gold:
    eid = g.get("expediente_id") or Path(str(g.get("file",""))).stem
    gold_map[eid] = extract_fields(g)

# Mapa predicciones v4
pred_map = {}
for p in pred_v4:
    eid = p.get("expediente_id") or Path(str(p.get("file",""))).stem
    pred_map[eid] = p.get("ie_fields_v4") or p.get("ie_fields") or {}

# --- Evaluación ---
rows=[]
for eid, ref_fields in gold_map.items():
    pred_fields = pred_map.get(eid, {})
    for c in CAMPOS:
        ref  = norm(ref_fields.get(c))
        pred = norm(pred_fields.get(c))
        em  = float(pred==ref)
        f1  = f1_str(pred, ref)
        rows.append({
            "expediente_id": eid,
            "campo": c,
            "pred": pred_fields.get(c),
            "ref": ref_fields.get(c),
            "EM": em,
            "F1": f1
        })

df = pd.DataFrame(rows)
resumen = df.groupby("campo")[["EM","F1"]].mean().reset_index()

print("=== MÉTRICAS POR CAMPO (Evaluación oficial) ===")
display(resumen)
print("\n=== RESUMEN GENERAL ===")
print(f"EM global: {df['EM'].mean():.3f} | F1 global: {df['F1'].mean():.3f}")

# --- Guardar resultados ---
OUT_DIR = Path("/content/outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)
resumen.to_csv(OUT_DIR/"resumen_metricas_v4.csv", index=False, encoding="utf-8")
df.to_csv(OUT_DIR/"errores_detalle_v4.csv", index=False, encoding="utf-8")
log(" Evaluación oficial guardada en /content/outputs")

# --- Comparativa con v3 (opcional) ---
if PRED_V3.exists():
    pred_v3 = read_jsonl(PRED_V3)
    pred3_map = {p.get("expediente_id"): p.get("ie_fields_v3") or p.get("ie_fields") or {} for p in pred_v3}

    rows3=[]
    for eid, ref_fields in gold_map.items():
        pred_fields = pred3_map.get(eid, {})
        for c in CAMPOS:
            ref = norm(ref_fields.get(c))
            pred = norm(pred_fields.get(c))
            rows3.append({"campo":c, "EM":float(pred==ref)})
    df3 = pd.DataFrame(rows3)
    comp = resumen.merge(
        df3.groupby("campo")["EM"].mean().reset_index().rename(columns={"EM":"EM_v3"}),
        on="campo", how="left"
    )
    comp["ΔEM_v4-v3"] = comp["EM"] - comp["EM_v3"]
    print("\n=== COMPARATIVA EM v3 vs v4 ===")
    display(comp)
    comp.to_csv(OUT_DIR/"comparativa_v3_v4.csv", index=False, encoding="utf-8")
    log(" Comparativa guardada en /content/outputs/comparativa_v3_v4.csv")
else:
    print("\n(No se encontró v3; se omitió comparativa.)")


=== MÉTRICAS POR CAMPO (Evaluación oficial) ===


,campo,EM,F1
0,articulos_citados,0.033333,0.033333
1,decision,0.000000,0.000000
2,fecha_acto,0.000000,0.000000
3,fiscal,0.000000,0.000000
4,juez,0.000000,0.000000
5,numero_proceso,0.000000,0.000000
6,tipo_acto,0.000000,0.000000



=== RESUMEN GENERAL ===
EM global: 0.005 | F1 global: 0.005
[08:47:03]  Evaluación oficial guardada en /content/outputs

=== COMPARATIVA EM v3 vs v4 ===


,campo,EM,F1,EM_v3,ΔEM_v4-v3
0,articulos_citados,0.033333,0.033333,0.066667,-0.033333
1,decision,0.000000,0.000000,0.000000,0.000000
2,fecha_acto,0.000000,0.000000,0.000000,0.000000
3,fiscal,0.000000,0.000000,0.000000,0.000000
4,juez,0.000000,0.000000,0.000000,0.000000
5,numero_proceso,0.000000,0.000000,0.000000,0.000000
6,tipo_acto,0.000000,0.000000,0.000000,0.000000


[08:47:03]  Comparativa guardada en /content/outputs/comparativa_v3_v4.csv


# Conclusiones

La arquitectura RAG híbrida con generador T5 y apoyos regex/NER deja una tubería reproducible, medible y reanudable, con artefactos versionados (Parquet, JSONL, CSV). Los índices BM25+E5 recuperan bien pasajes en “SENTENCIA/PROVIDENCIA”, pero la extracción campo-a-campo falla ante ruido de PDF, variabilidad de formato y valores que no están literalmente en el contexto o aparecen dispersos. La extensión con re-ranking CrossEncoder y NER no mejoró métricas en este lote, lo que sugiere que el cuello de botella está en parseo/OCR y en el formateo heterogéneo más que en la priorización de pasajes.

# Recomendaciones

Sustituir el parser por uno layout-aware con OCR robusto (p. ej., pdfminer + un OCR de alta calidad y un detector de bloques) y normalización canónica de cabeceras, para estabilizar numero_proceso/fecha_acto/juez/fiscal; en tus errores, muchas predicciones fueron None por simple ausencia de patrón fiable. Introducir reglas deterministas específicas de providencias y sentencias (plantillas con expresiones normalizadas por juzgado y época) y una tabla de sinónimos para títulos de sección. Cuando el preprocesamiento esté saneado, considerar un ajuste ligero PEFT/QLoRA del generador y/o un NER jurídico entrenado en ejemplos reales; con 30 expedientes no es suficiente para FT amplio, pero sí para calibrar un cabezal de span-classification y mejorar artículos_citados y autoridades.